1. Extract all data from CSV files.
2. Transform data into inputs/targets.
3. Split inputs/targets into train/test sets.
4. Sample inputs/targets for the testing sets.
5. Load outputs inputs/targets to parquet files.

# Imports 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split